In [1]:
from IPython.extensions import autoreload
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

In [3]:
header = ['user_id','item_id','rating','timestamp']
df = pd.read_csv('CF/u.data',sep='\t',names=header)

In [8]:
n_users = df['user_id'].unique().shape[0]
n_items = df['item_id'].unique().shape[0]

In [11]:
from sklearn import cross_validation as cv
train_data,test_data = cv.train_test_split(df,test_size=0.2)

D:\Python\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
train_data_matrix = np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1,line[2] - 1] = line[3]
print('train_data_matrix\n%s' % train_data_matrix)

train_data_matrix
[[5. 3. 4. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [13]:
test_data_matrix = np.zeros((n_users,n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1,line[2] - 1] = line[3]
print('test_data_matrix\n%s' % test_data_matrix)    

test_data_matrix
[[0. 0. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = pairwise_distances(train_data_matrix.T,metric='cosine')

In [15]:
def predict(ratings,similarity,type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:,np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [17]:
user_similarity = pairwise_distances(train_data_matrix,metric='cosine')
item_prediction = predict(train_data_matrix,item_similarity,type='item')
user_prediction = predict(train_data_matrix,user_similarity,type='user')
print(item_prediction[0:8])

[[0.37911317 0.39964562 0.41334468 ... 0.46741353 0.45747415 0.45419917]
 [0.09336584 0.10585406 0.10132879 ... 0.10502026 0.10667181 0.10757476]
 [0.06256774 0.06477381 0.06244508 ... 0.05903129 0.0635351  0.0638855 ]
 ...
 [0.30724496 0.34114053 0.35122093 ... 0.37542354 0.36900091 0.36893046]
 [0.64257152 0.66244407 0.69852073 ... 0.74736216 0.73340921 0.73265996]
 [0.08507014 0.08889779 0.10018164 ... 0.11479227 0.11042778 0.11180355]]


In [18]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction,ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,ground_truth))

print('Item-base CF RMSE: ' + str(rmse(item_prediction,test_data_matrix)))

Item-base CF RMSE: 3.4348264202881267
